In [ ]:
# import pandas as pd
# import numpy as np
# import requests
# from decimal import Decimal, ROUND_DOWN
# import json
# from datetime import datetime, timedelta

# # Global variables
# account_balance = 10000.0  # Starting balance ($10,000)
# open_positions = []
# total_profit = 0.0
# max_balance = account_balance
# daily_returns = []
# max_trade_size = 1.0  # Maximum trade size in BTC

# # Function to round trade size to 5 decimal places
# def round_trade_size(size):
#     return float(Decimal(str(size)).quantize(Decimal('0.00001'), rounding=ROUND_DOWN))

# # Function to fetch funding rate data from API
# def fetch_funding_rate_data(start_date, end_date):
#     """
#     Fetch funding rate data from CryptoQuant API for Binance and Bybit
#     """
#     # Format dates for API request
#     start_str = start_date.strftime("%Y%m%d")
    
#     # Make API request
#     url = f"https://api.cryptoquant.com/v1/btc/market-data/funding-rates?window=day&from={start_str}&exchange=binance,bybit"
    
#     # Note: In a real implementation, you would need to add your API key
#     # headers = {"Authorization": "Bearer YOUR_API_KEY"}
    
#     # For demonstration purposes, let's create sample data
#     # In a real implementation, you would use:
#     # response = requests.get(url, headers=headers)
#     # data = response.json()
    
#     # Create synthetic data for demonstration
#     days = (end_date - start_date).days
    
#     # Create timestamps for each day
#     timestamps = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(days)]
    
#     # Generate synthetic funding rates with some volatility
#     binance_rates = np.random.normal(0.01, 0.02, days)  # Mean of 0.01 (1%), std dev of 0.02 (2%)
#     bybit_rates = np.random.normal(0.0095, 0.018, days)  # Slightly different distribution
    
#     # Create synthetic price data
#     btc_price = 65000  # Starting price
#     btc_prices = []
    
#     for i in range(days):
#         # Random daily price change between -5% and 5%
#         daily_change = np.random.uniform(-0.05, 0.05)
#         btc_price = btc_price * (1 + daily_change)
#         btc_prices.append(btc_price)
    
#     # Create DataFrame
#     df = pd.DataFrame({
#         'timestamp': timestamps,
#         'binance_funding_rate': binance_rates,
#         'bybit_funding_rate': bybit_rates,
#         'close_spot': btc_prices,  # Use same price for spot and futures for simplicity
#         'close_futures': btc_prices
#     })
    
#     # Calculate funding rate difference (opportunity for arbitrage)
#     df['funding_rate'] = df['binance_funding_rate'] - df['bybit_funding_rate']
    
#     return df

# # Function to calculate Sharpe ratio
# def calculate_sharpe_ratio(returns, risk_free_rate, trading_days=365):
#     """
#     Calculate the annualized Sharpe ratio.
    
#     Parameters:
#     - returns: List of daily returns
#     - risk_free_rate: Annual risk-free rate (e.g., 0.02 for 2%)
#     - trading_days: Number of trading days in a year
    
#     Returns:
#     - Sharpe ratio
#     """
#     if not returns or len(returns) < 2:
#         return 0
    
#     # Convert annual risk-free rate to daily
#     daily_risk_free = (1 + risk_free_rate) ** (1 / trading_days) - 1
    
#     # Calculate excess returns
#     excess_returns = [r - daily_risk_free for r in returns]
    
#     # Calculate mean and standard deviation of excess returns
#     mean_excess_return = np.mean(excess_returns)
#     std_excess_return = np.std(excess_returns, ddof=1)  # Sample standard deviation
    
#     if std_excess_return == 0:
#         return 0
    
#     # Daily Sharpe ratio
#     daily_sharpe = mean_excess_return / std_excess_return
    
#     # Annualized Sharpe ratio
#     annualized_sharpe = daily_sharpe * np.sqrt(trading_days)
    
#     return annualized_sharpe

# # Function to check if arbitrage opportunity exists
# def check_arbitrage_opportunity(df, min_funding_rate):
#     # Filter out opportunities that are too small
#     return df[abs(df['funding_rate']) >= min_funding_rate]

# # Function to size position based on funding rate magnitude
# def calculate_position_size(funding_rate, trade_size, max_trade_size, min_funding_rate):
#     # Scale position size based on funding rate magnitude (larger funding = larger position)
#     # But never exceed max_trade_size
#     scaled_size = trade_size * (abs(funding_rate) / min_funding_rate)
#     # Round to 5 decimal places
#     return round_trade_size(min(scaled_size, max_trade_size))

# # Function to determine hedge positions
# def open_hedge_position(trade_size, funding_rate, mark_price):
#     if funding_rate > 0:  # Binance has higher funding rate
#         exchange_high = "Binance"
#         exchange_low = "Bybit"
#         side_spot = 'buy'
#         side_futures = 'sell'
#     else:  # Bybit has higher funding rate
#         exchange_high = "Bybit"
#         exchange_low = "Binance"
#         side_spot = 'sell'
#         side_futures = 'buy'
    
#     position_value = trade_size * mark_price
#     print(f"Opened hedge positions: {side_futures} futures on {exchange_high} and {side_spot} spot on {exchange_low}, Size: {trade_size} BTC (${position_value:.2f})")
#     return side_spot, side_futures

# # Function to close positions
# def close_positions(trade_size, funding_rate, mark_price):
#     if funding_rate > 0:  # Binance had higher funding rate
#         exchange_high = "Binance"
#         exchange_low = "Bybit"
#         side_spot = 'sell'
#         side_futures = 'buy'
#     else:  # Bybit had higher funding rate
#         exchange_high = "Bybit"
#         exchange_low = "Binance"
#         side_spot = 'buy'
#         side_futures = 'sell'
    
#     position_value = trade_size * mark_price
#     print(f"Closed hedge positions: {side_spot} spot on {exchange_low} and {side_futures} futures on {exchange_high}, Size: {trade_size} BTC (${position_value:.2f})")
#     return side_spot, side_futures

# # Function to calculate total profit
# def calculate_total_profit(trade_size, funding_rate, mark_price, entry_spot, close_spot, entry_futures, close_futures):
#     """
#     Calculates total funding arbitrage profit:
#     - Funding fee profit
#     - Spot position profit/loss
#     - Futures position profit/loss
#     """
#     # Funding Profit (8-hour funding period)
#     funding_profit = trade_size * abs(funding_rate) * mark_price

#     # Determine trade direction based on funding rate
#     if funding_rate > 0:
#         spot_side_multiplier = 1      # Buying spot (long)
#         futures_side_multiplier = -1  # Shorting futures
#     else:
#         spot_side_multiplier = -1     # Selling spot (short)
#         futures_side_multiplier = 1   # Longing futures

#     # Spot Profit/Loss
#     spot_pnl = trade_size * (close_spot - entry_spot) * spot_side_multiplier

#     # Futures Profit/Loss
#     futures_pnl = trade_size * (entry_futures - close_futures) * futures_side_multiplier

#     # Total Profit
#     total_profit = funding_profit + spot_pnl + futures_pnl

#     return total_profit, funding_profit, spot_pnl, futures_pnl

# # Check if stop loss is triggered
# def check_stop_loss(position, current_spot, current_futures, stop_loss_pct):
#     funding_rate = position['funding_rate']
#     entry_spot = position['entry_spot']
#     entry_futures = position['entry_futures']
#     trade_size = position['trade_size']
    
#     # Calculate current PnL (excluding funding, which is already locked in)
#     if funding_rate > 0:  # Long spot, short futures
#         spot_pnl = trade_size * (current_spot - entry_spot)
#         futures_pnl = trade_size * (entry_futures - current_futures)
#     else:  # Short spot, long futures
#         spot_pnl = trade_size * (entry_spot - current_spot)
#         futures_pnl = trade_size * (current_futures - entry_futures)
    
#     current_pnl = spot_pnl + futures_pnl
#     position_value = trade_size * entry_spot
    
#     # Check if loss exceeds stop loss threshold
#     if current_pnl < -position_value * stop_loss_pct:
#         return True
#     return False

# # Function to run the funding arbitrage bot with risk management
# def funding_arbitrage_bot(df, trade_size, min_funding_rate, max_open_positions, stop_loss_pct, max_drawdown_pct, risk_free_rate):
#     global account_balance, open_positions, total_profit, max_balance, daily_returns
    
#     results = []
#     position_id = 0
    
#     # Add date column for grouping returns by day
#     df['date'] = pd.to_datetime(df['timestamp']).dt.date
    
#     # Create a daily balance tracker for Sharpe ratio calculation
#     daily_balance = {}
#     prev_balance = account_balance
#     current_date = None
    
#     # Filter for arbitrage opportunities
#     arbitrage_df = check_arbitrage_opportunity(df, min_funding_rate)

#     for index, row in df.iterrows():
#         timestamp = row['timestamp']
#         current_date = row['date']
#         current_spot = row['close_spot']
#         current_futures = row['close_futures']
        
#         # 1. Check existing positions for stop loss or closing
#         positions_to_close = []
#         for pos in open_positions:
#             # Check if stop loss is triggered
#             if check_stop_loss(pos, current_spot, current_futures, stop_loss_pct):
#                 print(f"\n[Position {pos['id']}] ⚠️ Stop loss triggered! Closing position.")
#                 positions_to_close.append(pos)
            
#             # Check if position has reached its target close time
#             elif index >= pos['target_close_index']:
#                 print(f"\n[Position {pos['id']}] Target close time reached. Closing position.")
#                 positions_to_close.append(pos)
        
#         # Close positions that need closing
#         for pos in positions_to_close:
#             close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
            
#             # Calculate final profit
#             profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
#                 pos['trade_size'], pos['funding_rate'], pos['mark_price'],
#                 pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
#             )
            
#             # Update account balance
#             account_balance += profit
#             total_profit += profit
#             max_balance = max(max_balance, account_balance)
            
#             # Log results
#             results.append([
#                 timestamp, pos['funding_rate'], pos['trade_size'], 
#                 pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
#                 profit, funding_profit, spot_pnl, futures_pnl
#             ])
            
#             print(f"Position closed. Profit: ${profit:.2f} (Funding: ${funding_profit:.2f}, Spot: ${spot_pnl:.2f}, Futures: ${futures_pnl:.2f})")
#             open_positions.remove(pos)
        
#         # 2. Check drawdown - pause trading if exceeded
#         current_drawdown = (max_balance - account_balance) / max_balance if max_balance > 0 else 0
#         if current_drawdown > max_drawdown_pct:
#             print(f"⚠️ Max drawdown exceeded ({current_drawdown:.2%}). Pausing new trades.")
#             continue
        
#         # 3. Check for new arbitrage opportunity
#         funding_rate = row['funding_rate']
#         if abs(funding_rate) >= min_funding_rate:
            
#             # Check if we can open new positions
#             if len(open_positions) < max_open_positions:
#                 mark_price = row['close_futures']  # Mark price is usually close to futures price
#                 entry_spot = row['close_spot']
#                 entry_futures = row['close_futures']
                
#                 # Size position based on funding rate magnitude
#                 adjusted_size = calculate_position_size(funding_rate, trade_size, max_trade_size, min_funding_rate)
                
#                 # Calculate capital needed
#                 position_value = adjusted_size * mark_price
                
#                 # Check if we have enough capital (using 50% of available balance per trade)
#                 if position_value <= account_balance * 0.5:
#                     print(f"\n[New Position {position_id}] Arbitrage Opportunity Found! Funding Rate Difference: {funding_rate:.6f}")
                    
#                     # Open hedge positions
#                     side_spot, side_futures = open_hedge_position(adjusted_size, funding_rate, mark_price)
                    
#                     # Target closing at next funding time (assuming daily funding in this case)
#                     target_close_index = index + 1
                    
#                     # Record position
#                     position = {
#                         'id': position_id,
#                         'timestamp': timestamp,
#                         'funding_rate': funding_rate,
#                         'trade_size': adjusted_size,
#                         'entry_spot': entry_spot,
#                         'entry_futures': entry_futures,
#                         'mark_price': mark_price,
#                         'open_index': index,
#                         'target_close_index': target_close_index
#                     }
#                     open_positions.append(position)
#                     position_id += 1
#                 else:
#                     print(f"Insufficient balance for new position. Required: ${position_value:.2f}, Available: ${account_balance:.2f}")
        
#         # Track daily balance for returns calculation
#         if current_date not in daily_balance:
#             # If this is a new day, calculate return from previous day
#             if prev_balance > 0 and len(daily_balance) > 0:
#                 # Calculate daily return
#                 daily_return = (account_balance / prev_balance) - 1
#                 daily_returns.append(daily_return)
                
#             # Store today's balance
#             daily_balance[current_date] = account_balance
#             prev_balance = account_balance
    
#     # Close any remaining open positions at the end of the backtest
#     for pos in open_positions:
#         current_spot = df.iloc[-1]['close_spot']
#         current_futures = df.iloc[-1]['close_futures']
        
#         close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
        
#         # Calculate final profit
#         profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
#             pos['trade_size'], pos['funding_rate'], pos['mark_price'],
#             pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
#         )
        
#         # Update account balance
#         account_balance += profit
#         total_profit += profit
        
#         # Log results
#         results.append([
#             df.iloc[-1]['timestamp'], pos['funding_rate'], pos['trade_size'], 
#             pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
#             profit, funding_profit, spot_pnl, futures_pnl
#         ])

#     # Final balance for the last day
#     if current_date and prev_balance > 0:
#         daily_return = (account_balance / prev_balance) - 1
#         daily_returns.append(daily_return)

#     # Calculate Sharpe ratio
#     # Trading days per year is 365 for daily data
#     trading_periods_per_year = 365  # For daily data
#     sharpe_ratio = calculate_sharpe_ratio(daily_returns, risk_free_rate, trading_periods_per_year)

#     # Save results to CSV
#     results_df = pd.DataFrame(results, columns=[
#         'timestamp', 'funding_rate', 'trade_size', 'entry_spot', 'close_spot', 
#         'entry_futures', 'close_futures', 'total_profit', 'funding_profit', 
#         'spot_pnl', 'futures_pnl'
#     ])
#     results_df.to_csv("cross_ex_funding_arbitrage_results.csv", index=False)
    
#     # Save daily returns for further analysis
#     daily_returns_df = pd.DataFrame({
#         'date': list(daily_balance.keys()),
#         'balance': list(daily_balance.values())
#     })
#     daily_returns_df['cross_ex_daily_return'] = daily_returns_df['balance'].pct_change()
#     daily_returns_df.to_csv("cross_ex_daily_returns.csv", index=False)
    
#     # Calculate additional performance metrics
#     total_trades = len(results_df)
#     profitable_trades = len(results_df[results_df['total_profit'] > 0])
#     win_rate = profitable_trades / total_trades if total_trades > 0 else 0
#     avg_profit = results_df['total_profit'].mean() if total_trades > 0 else 0
    
#     # Calculate volatility (standard deviation of returns)
#     volatility = np.std(daily_returns, ddof=1) * np.sqrt(trading_periods_per_year) if daily_returns else 0
    
#     # Calculate max drawdown
#     cumulative_returns = np.cumprod(np.array([1 + r for r in daily_returns]))
#     running_max = np.maximum.accumulate(cumulative_returns)
#     drawdowns = (cumulative_returns / running_max) - 1
#     max_dd = abs(np.min(drawdowns)) if len(drawdowns) > 0 else 0
    
#     # Calculate annualized return
#     if len(daily_returns) > 0:
#         cagr = ((account_balance / 10000) ** (365 / len(daily_balance))) - 1
#     else:
#         cagr = 0
    
#     print("\n=== PERFORMANCE SUMMARY ===")
#     print(f"Starting Balance: ${10000:.2f}")
#     print(f"Final Balance: ${account_balance:.2f}")
#     print(f"Total Profit: ${total_profit:.2f} ({(account_balance/10000 - 1)*100:.2f}%)")
#     print(f"Trading Days: {len(daily_balance)}")
#     print(f"Annualized Return: {cagr:.2%}")
#     print(f"Annualized Volatility: {volatility:.2%}")
#     print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
#     print(f"Max Drawdown: {max_dd:.2%}")
#     print(f"Total Trades: {total_trades}")
#     print(f"Win Rate: {win_rate:.2%}")
#     print(f"Average Profit per Trade: ${avg_profit:.2f}")
    
#     # Analyze performance by funding rate ranges
#     if total_trades > 0:
#         results_df['funding_rate_abs'] = results_df['funding_rate'].abs()
#         results_df['funding_range'] = pd.cut(results_df['funding_rate_abs'], 
#                                             bins=[0, 0.001, 0.003, 0.005, 0.01, 0.05, 1], 
#                                             labels=['0-0.1%', '0.1-0.3%', '0.3-0.5%', '0.5-1%', '1-5%', '>5%'])
#         funding_performance = results_df.groupby('funding_range')['total_profit'].agg(['mean', 'count'])
#         print("\nPerformance by Funding Rate Range:")
#         print(funding_performance)
    
#     print("\n✅ All trades completed. Results saved to 'cross_ex_funding_arbitrage_results.csv'.")
#     return sharpe_ratio, cagr, volatility, max_dd

# # Main function to run the backtest
# def main():
#     # Set parameters
#     trade_size = 0.1  # Base trade size in BTC
#     min_funding_rate = 0.0005  # Minimum funding rate difference (0.05%)
#     max_open_positions = 5
#     stop_loss_pct = 0.03  # 3% stop loss
#     max_drawdown_pct = 0.20  # 20% max drawdown
#     risk_free_rate = 0.04  # 4% risk-free rate
    
#     # Set date range for backtest
#     start_date = datetime(2024, 3, 1)
#     end_date = datetime(2025, 3, 1)
    
#     # Fetch data from API (or simulate it in this case)
#     print("Fetching funding rate data...")
#     df = fetch_funding_rate_data(start_date, end_date)
#     print(f"Data fetched. {len(df)} data points available.")
    
#     # Run the backtest
#     print("\n=== STARTING FUNDING ARBITRAGE BACKTEST ===")
#     print(f"Time period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
#     print(f"Min Funding Rate: {min_funding_rate:.4f}")
#     print(f"Stop Loss: {stop_loss_pct:.1%}")
#     print(f"Max Drawdown: {max_drawdown_pct:.1%}")
    
#     # Run the bot
#     sharpe, cagr, volatility, max_dd = funding_arbitrage_bot(
#         df, trade_size, min_funding_rate, max_open_positions, 
#         stop_loss_pct, max_drawdown_pct, risk_free_rate
#     )
    
#     # Return key metrics
#     return {
#         "Sharpe Ratio": sharpe,
#         "CAGR": cagr,
#         "Volatility": volatility,
#         "Max Drawdown": max_dd,
#         "Final Balance": account_balance
#     }

# if __name__ == "__main__":
#     main()

Fetching funding rate data...
Data fetched. 365 data points available.

=== STARTING FUNDING ARBITRAGE BACKTEST ===
Time period: 2024-03-01 to 2025-03-01
Min Funding Rate: 0.0005
Stop Loss: 3.0%
Max Drawdown: 20.0%
Insufficient balance for new position. Required: $64025.49, Available: $10000.00
Insufficient balance for new position. Required: $59763.38, Available: $10000.00
Insufficient balance for new position. Required: $58984.57, Available: $10000.00
Insufficient balance for new position. Required: $57639.73, Available: $10000.00
Insufficient balance for new position. Required: $57396.39, Available: $10000.00
Insufficient balance for new position. Required: $56548.34, Available: $10000.00
Insufficient balance for new position. Required: $58483.44, Available: $10000.00
Insufficient balance for new position. Required: $50031.85, Available: $10000.00
Insufficient balance for new position. Required: $56250.55, Available: $10000.00
Insufficient balance for new position. Required: $54934.

In [2]:
import pandas as pd
import numpy as np
import requests
from decimal import Decimal, ROUND_DOWN
import json
from datetime import datetime, timedelta

# Global variables
account_balance = 10000.0  # Starting balance ($10,000)
open_positions = []
total_profit = 0.0
max_balance = account_balance
daily_returns = []
min_trade_size = 0.01  # Minimum trade size in BTC
max_trade_size = 0.05  # Maximum trade size in BTC

# Function to round trade size to 5 decimal places
def round_trade_size(size):
    return float(Decimal(str(size)).quantize(Decimal('0.00001'), rounding=ROUND_DOWN))

# Function to fetch funding rate data from API
def fetch_funding_rate_data(start_date, end_date):
    """
    Fetch funding rate data from CryptoQuant API for Binance and Bybit
    """
    # Format dates for API request
    start_str = start_date.strftime("%Y%m%d")
    
    # Make API request
    url = f"https://api.cryptoquant.com/v1/btc/market-data/funding-rates?window=day&from={start_str}&exchange=binance,bybit"
    
    # Note: In a real implementation, you would need to add your API key
    # headers = {"Authorization": "Bearer YOUR_API_KEY"}
    
    # For demonstration purposes, let's create sample data with more realistic funding rates
    # In a real implementation, you would use:
    # response = requests.get(url, headers=headers)
    # data = response.json()
    
    # Create synthetic data for demonstration
    days = (end_date - start_date).days
    
    # Create timestamps for each day
    timestamps = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(days)]
    
    # Generate synthetic funding rates with realistic values
    binance_rates = np.random.normal(0.0008, 0.0015, days)  # Mean of 0.08%, std dev of 0.15%
    bybit_rates = np.random.normal(0.0007, 0.0013, days)  # Slightly different distribution
    
    # Fetch real historical BTC price data using CoinGecko API
    try:
        print("Fetching historical Bitcoin price data...")
        # Convert dates to unix timestamps for CoinGecko
        from_timestamp = int(start_date.timestamp())
        to_timestamp = int(end_date.timestamp())
        
        # CoinGecko API endpoint for Bitcoin price history
        coin_gecko_url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from={from_timestamp}&to={to_timestamp}"
        response = requests.get(coin_gecko_url)
        
        if response.status_code == 200:
            price_data = response.json()
            prices = [price[1] for price in price_data['prices']]  # Extract just the prices
            
            # If we have more price data than days, sample down
            if len(prices) > days:
                # Take prices at regular intervals to match our days
                btc_prices = [prices[int(i * len(prices) / days)] for i in range(days)]
            else:
                # If we have fewer prices, we'll need to interpolate
                # For simplicity, we'll just repeat the last price
                btc_prices = prices + [prices[-1]] * (days - len(prices))
            
            print(f"Successfully fetched {len(btc_prices)} Bitcoin price data points")
        else:
            print(f"Failed to fetch Bitcoin price data: {response.status_code}")
            # Fallback to a realistic starting price with some volatility
            btc_price = 65000  # Starting price
            btc_prices = []
            for i in range(days):
                daily_change = np.random.uniform(-0.03, 0.03)  # More realistic daily fluctuation
                btc_price = btc_price * (1 + daily_change)
                btc_prices.append(btc_price)
            print("Using synthetic Bitcoin price data as fallback")
    except Exception as e:
        print(f"Error fetching Bitcoin price data: {e}")
        # Fallback to synthetic data
        btc_price = 65000  # Starting price
        btc_prices = []
        for i in range(days):
            daily_change = np.random.uniform(-0.03, 0.03)
            btc_price = btc_price * (1 + daily_change)
            btc_prices.append(btc_price)
        print("Using synthetic Bitcoin price data as fallback")
    
    # Create DataFrame
    df = pd.DataFrame({
        'timestamp': timestamps,
        'binance_funding_rate': binance_rates,
        'bybit_funding_rate': bybit_rates,
        'close_spot': btc_prices,  # Use real price data
        'close_futures': btc_prices  # For simplicity, use same price
    })
    
    # Calculate funding rate difference (opportunity for arbitrage)
    df['funding_rate'] = df['binance_funding_rate'] - df['bybit_funding_rate']
    
    return df

# Function to calculate Sharpe ratio
def calculate_sharpe_ratio(returns, risk_free_rate, trading_days=365):
    """
    Calculate the annualized Sharpe ratio.
    
    Parameters:
    - returns: List of daily returns
    - risk_free_rate: Annual risk-free rate (e.g., 0.02 for 2%)
    - trading_days: Number of trading days in a year
    
    Returns:
    - Sharpe ratio
    """
    if not returns or len(returns) < 2:
        return 0
    
    # Convert annual risk-free rate to daily
    daily_risk_free = (1 + risk_free_rate) ** (1 / trading_days) - 1
    
    # Calculate excess returns
    excess_returns = [r - daily_risk_free for r in returns]
    
    # Calculate mean and standard deviation of excess returns
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns, ddof=1)  # Sample standard deviation
    
    if std_excess_return == 0:
        return 0
    
    # Daily Sharpe ratio
    daily_sharpe = mean_excess_return / std_excess_return
    
    # Annualized Sharpe ratio
    annualized_sharpe = daily_sharpe * np.sqrt(trading_days)
    
    return annualized_sharpe

# Function to check if arbitrage opportunity exists
def check_arbitrage_opportunity(df, min_funding_rate):
    # Filter out opportunities that are too small
    return df[abs(df['funding_rate']) >= min_funding_rate]

# Function to size position based on funding rate magnitude
def calculate_position_size(funding_rate, min_funding_rate, min_size=0.01, max_size=0.05):
    """
    Scale position size between min_size and max_size based on funding rate magnitude.
    Ensures a minimum size of 0.01 BTC and a maximum of 0.05 BTC.
    
    Parameters:
    - funding_rate: Current funding rate difference
    - min_funding_rate: Minimum funding rate to consider for arbitrage
    - min_size: Minimum trade size (default: 0.01 BTC)
    - max_size: Maximum trade size (default: 0.05 BTC)
    
    Returns:
    - Sized position rounded to 5 decimal places
    """
    # Normalize the funding rate (how much larger is it than our minimum threshold)
    # The scale factor ranges from 1.0 to 5.0 as funding_rate increases
    normalized_rate = abs(funding_rate) / min_funding_rate
    
    # Cap the normalized rate to avoid extremely large positions
    capped_normalized = min(normalized_rate, 5.0)
    
    # Linear scaling between min_size and max_size
    scaled_size = min_size + (capped_normalized - 1) * (max_size - min_size) / 4
    
    # Ensure we're between min and max trade size
    scaled_size = max(min_size, min(scaled_size, max_size))
    
    # Round to 5 decimal places
    return round_trade_size(scaled_size)

# Function to determine hedge positions
def open_hedge_position(trade_size, funding_rate, mark_price):
    if funding_rate > 0:  # Binance has higher funding rate
        exchange_high = "Binance"
        exchange_low = "Bybit"
        side_spot = 'buy'
        side_futures = 'sell'
    else:  # Bybit has higher funding rate
        exchange_high = "Bybit"
        exchange_low = "Binance"
        side_spot = 'sell'
        side_futures = 'buy'
    
    position_value = trade_size * mark_price
    print(f"Opened hedge positions: {side_futures} futures on {exchange_high} and {side_spot} spot on {exchange_low}, Size: {trade_size} BTC (${position_value:.2f})")
    return side_spot, side_futures

# Function to close positions
def close_positions(trade_size, funding_rate, mark_price):
    if funding_rate > 0:  # Binance had higher funding rate
        exchange_high = "Binance"
        exchange_low = "Bybit"
        side_spot = 'sell'
        side_futures = 'buy'
    else:  # Bybit had higher funding rate
        exchange_high = "Bybit"
        exchange_low = "Binance"
        side_spot = 'buy'
        side_futures = 'sell'
    
    position_value = trade_size * mark_price
    print(f"Closed hedge positions: {side_spot} spot on {exchange_low} and {side_futures} futures on {exchange_high}, Size: {trade_size} BTC (${position_value:.2f})")
    return side_spot, side_futures

# Function to calculate total profit
def calculate_total_profit(trade_size, funding_rate, mark_price, entry_spot, close_spot, entry_futures, close_futures):
    """
    Calculates total funding arbitrage profit:
    - Funding fee profit
    - Spot position profit/loss
    - Futures position profit/loss
    """
    # Funding Profit (8-hour funding period)
    funding_profit = trade_size * abs(funding_rate) * mark_price

    # Determine trade direction based on funding rate
    if funding_rate > 0:
        spot_side_multiplier = 1      # Buying spot (long)
        futures_side_multiplier = -1  # Shorting futures
    else:
        spot_side_multiplier = -1     # Selling spot (short)
        futures_side_multiplier = 1   # Longing futures

    # Spot Profit/Loss
    spot_pnl = trade_size * (close_spot - entry_spot) * spot_side_multiplier

    # Futures Profit/Loss
    futures_pnl = trade_size * (entry_futures - close_futures) * futures_side_multiplier

    # Total Profit
    total_profit = funding_profit + spot_pnl + futures_pnl

    return total_profit, funding_profit, spot_pnl, futures_pnl

# Check if stop loss is triggered
def check_stop_loss(position, current_spot, current_futures, stop_loss_pct):
    funding_rate = position['funding_rate']
    entry_spot = position['entry_spot']
    entry_futures = position['entry_futures']
    trade_size = position['trade_size']
    
    # Calculate current PnL (excluding funding, which is already locked in)
    if funding_rate > 0:  # Long spot, short futures
        spot_pnl = trade_size * (current_spot - entry_spot)
        futures_pnl = trade_size * (entry_futures - current_futures)
    else:  # Short spot, long futures
        spot_pnl = trade_size * (entry_spot - current_spot)
        futures_pnl = trade_size * (current_futures - entry_futures)
    
    current_pnl = spot_pnl + futures_pnl
    position_value = trade_size * entry_spot
    
    # Check if loss exceeds stop loss threshold
    if current_pnl < -position_value * stop_loss_pct:
        return True
    return False

# Function to run the funding arbitrage bot with risk management
def funding_arbitrage_bot(df, min_funding_rate, max_open_positions, stop_loss_pct, max_drawdown_pct, risk_free_rate):
    global account_balance, open_positions, total_profit, max_balance, daily_returns
    
    results = []
    position_id = 0
    
    # Add date column for grouping returns by day
    df['date'] = pd.to_datetime(df['timestamp']).dt.date
    
    # Create a daily balance tracker for Sharpe ratio calculation
    daily_balance = {}
    prev_balance = account_balance
    current_date = None
    
    # Filter for arbitrage opportunities
    arbitrage_df = check_arbitrage_opportunity(df, min_funding_rate)
    print(f"Found {len(arbitrage_df)} potential arbitrage opportunities")

    for index, row in df.iterrows():
        timestamp = row['timestamp']
        current_date = row['date']
        current_spot = row['close_spot']
        current_futures = row['close_futures']
        
        # 1. Check existing positions for stop loss or closing
        positions_to_close = []
        for pos in open_positions:
            # Check if stop loss is triggered
            if check_stop_loss(pos, current_spot, current_futures, stop_loss_pct):
                print(f"\n[Position {pos['id']}] ⚠️ Stop loss triggered! Closing position.")
                positions_to_close.append(pos)
            
            # Check if position has reached its target close time
            elif index >= pos['target_close_index']:
                print(f"\n[Position {pos['id']}] Target close time reached. Closing position.")
                positions_to_close.append(pos)
        
        # Close positions that need closing
        for pos in positions_to_close:
            close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
            
            # Calculate final profit
            profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
                pos['trade_size'], pos['funding_rate'], pos['mark_price'],
                pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
            )
            
            # Update account balance
            account_balance += profit
            total_profit += profit
            max_balance = max(max_balance, account_balance)
            
            # Log results
            results.append([
                timestamp, pos['funding_rate'], pos['trade_size'], 
                pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
                profit, funding_profit, spot_pnl, futures_pnl
            ])
            
            print(f"Position closed. Profit: ${profit:.2f} (Funding: ${funding_profit:.2f}, Spot: ${spot_pnl:.2f}, Futures: ${futures_pnl:.2f})")
            open_positions.remove(pos)
        
        # 2. Check drawdown - pause trading if exceeded
        current_drawdown = (max_balance - account_balance) / max_balance if max_balance > 0 else 0
        if current_drawdown > max_drawdown_pct:
            print(f"⚠️ Max drawdown exceeded ({current_drawdown:.2%}). Pausing new trades.")
            continue
        
        # 3. Check for new arbitrage opportunity
        funding_rate = row['funding_rate']
        if abs(funding_rate) >= min_funding_rate:
            
            # Check if we can open new positions
            if len(open_positions) < max_open_positions:
                mark_price = row['close_futures']  # Mark price is usually close to futures price
                entry_spot = row['close_spot']
                entry_futures = row['close_futures']
                
                # Size position based on funding rate magnitude
                adjusted_size = calculate_position_size(funding_rate, min_funding_rate, min_trade_size, max_trade_size)
                
                # Calculate capital needed for the position
                position_value = adjusted_size * mark_price
                
                # Check if we have enough capital (using 10% of available balance per trade)
                # Reduced from 50% to 10% to allow for more trades to be taken
                if position_value <= account_balance * 0.1:
                    print(f"\n[New Position {position_id}] Arbitrage Opportunity Found! Funding Rate Difference: {funding_rate:.6f}")
                    
                    # Open hedge positions
                    side_spot, side_futures = open_hedge_position(adjusted_size, funding_rate, mark_price)
                    
                    # Target closing at next funding time (assuming daily funding in this case)
                    target_close_index = index + 1
                    
                    # Record position
                    position = {
                        'id': position_id,
                        'timestamp': timestamp,
                        'funding_rate': funding_rate,
                        'trade_size': adjusted_size,
                        'entry_spot': entry_spot,
                        'entry_futures': entry_futures,
                        'mark_price': mark_price,
                        'open_index': index,
                        'target_close_index': target_close_index
                    }
                    open_positions.append(position)
                    position_id += 1
                else:
                    print(f"Insufficient balance for new position. Required: ${position_value:.2f}, Available: ${account_balance:.2f}")
        
        # Track daily balance for returns calculation
        if current_date not in daily_balance:
            # If this is a new day, calculate return from previous day
            if prev_balance > 0 and len(daily_balance) > 0:
                # Calculate daily return
                daily_return = (account_balance / prev_balance) - 1
                daily_returns.append(daily_return)
                
            # Store today's balance
            daily_balance[current_date] = account_balance
            prev_balance = account_balance
    
    # Close any remaining open positions at the end of the backtest
    for pos in open_positions:
        current_spot = df.iloc[-1]['close_spot']
        current_futures = df.iloc[-1]['close_futures']
        
        close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
        
        # Calculate final profit
        profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
            pos['trade_size'], pos['funding_rate'], pos['mark_price'],
            pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
        )
        
        # Update account balance
        account_balance += profit
        total_profit += profit
        
        # Log results
        results.append([
            df.iloc[-1]['timestamp'], pos['funding_rate'], pos['trade_size'], 
            pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
            profit, funding_profit, spot_pnl, futures_pnl
        ])

    # Final balance for the last day
    if current_date and prev_balance > 0:
        daily_return = (account_balance / prev_balance) - 1
        daily_returns.append(daily_return)

    # Calculate Sharpe ratio
    # Trading days per year is 365 for daily data
    trading_periods_per_year = 365  # For daily data
    sharpe_ratio = calculate_sharpe_ratio(daily_returns, risk_free_rate, trading_periods_per_year)

    # Save results to CSV
    results_df = pd.DataFrame(results, columns=[
        'timestamp', 'funding_rate', 'trade_size', 'entry_spot', 'close_spot', 
        'entry_futures', 'close_futures', 'total_profit', 'funding_profit', 
        'spot_pnl', 'futures_pnl'
    ])
    results_df.to_csv("cross_ex_funding_arbitrage_results.csv", index=False)
    
    # Save daily returns for further analysis
    daily_returns_df = pd.DataFrame({
        'date': list(daily_balance.keys()),
        'balance': list(daily_balance.values())
    })
    daily_returns_df['cross_ex_daily_return'] = daily_returns_df['balance'].pct_change()
    daily_returns_df.to_csv("cross_ex_daily_returns.csv", index=False)
    
    # Calculate additional performance metrics
    total_trades = len(results_df)
    profitable_trades = len(results_df[results_df['total_profit'] > 0])
    win_rate = profitable_trades / total_trades if total_trades > 0 else 0
    avg_profit = results_df['total_profit'].mean() if total_trades > 0 else 0
    
    # Calculate volatility (standard deviation of returns)
    volatility = np.std(daily_returns, ddof=1) * np.sqrt(trading_periods_per_year) if daily_returns else 0
    
    # Calculate max drawdown
    cumulative_returns = np.cumprod(np.array([1 + r for r in daily_returns]))
    running_max = np.maximum.accumulate(cumulative_returns)
    drawdowns = (cumulative_returns / running_max) - 1
    max_dd = abs(np.min(drawdowns)) if len(drawdowns) > 0 else 0
    
    # Calculate annualized return
    if len(daily_returns) > 0:
        cagr = ((account_balance / 10000) ** (365 / len(daily_balance))) - 1
    else:
        cagr = 0
    
    print("\n=== PERFORMANCE SUMMARY ===")
    print(f"Starting Balance: ${10000:.2f}")
    print(f"Final Balance: ${account_balance:.2f}")
    print(f"Total Profit: ${total_profit:.2f} ({(account_balance/10000 - 1)*100:.2f}%)")
    print(f"Trading Days: {len(daily_balance)}")
    print(f"Annualized Return: {cagr:.2%}")
    print(f"Annualized Volatility: {volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_dd:.2%}")
    print(f"Total Trades: {total_trades}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Average Profit per Trade: ${avg_profit:.2f}")
    
    # Analyze performance by funding rate ranges
    if total_trades > 0:
        results_df['funding_rate_abs'] = results_df['funding_rate'].abs()
        results_df['funding_range'] = pd.cut(results_df['funding_rate_abs'], 
                                            bins=[0, 0.001, 0.003, 0.005, 0.01, 0.05, 1], 
                                            labels=['0-0.1%', '0.1-0.3%', '0.3-0.5%', '0.5-1%', '1-5%', '>5%'])
        funding_performance = results_df.groupby('funding_range')['total_profit'].agg(['mean', 'count'])
        print("\nPerformance by Funding Rate Range:")
        print(funding_performance)
    
    print("\n✅ All trades completed. Results saved to 'cross_ex_funding_arbitrage_results.csv'.")
    return sharpe_ratio, cagr, volatility, max_dd

# Main function to run the backtest
def main():
    # Set parameters
    min_funding_rate = 0.0005  # Minimum funding rate difference (0.05%)
    max_open_positions = 5
    stop_loss_pct = 0.03  # 3% stop loss
    max_drawdown_pct = 0.20  # 20% max drawdown
    risk_free_rate = 0.04  # 4% risk-free rate
    
    # Set date range for backtest
    start_date = datetime(2024, 3, 1)
    end_date = datetime(2025, 3, 1)
    
    # Fetch data from API (or simulate it in this case)
    print("Fetching funding rate data...")
    df = fetch_funding_rate_data(start_date, end_date)
    print(f"Data fetched. {len(df)} data points available.")
    
    # Run the backtest
    print("\n=== STARTING FUNDING ARBITRAGE BACKTEST ===")
    print(f"Time period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"Min Funding Rate: {min_funding_rate:.4f}")
    print(f"Min Trade Size: {min_trade_size} BTC")
    print(f"Max Trade Size: {max_trade_size} BTC")
    print(f"Stop Loss: {stop_loss_pct:.1%}")
    print(f"Max Drawdown: {max_drawdown_pct:.1%}")
    
    # Run the bot
    sharpe, cagr, volatility, max_dd = funding_arbitrage_bot(
        df, min_funding_rate, max_open_positions, 
        stop_loss_pct, max_drawdown_pct, risk_free_rate
    )
    
    # Return key metrics
    return {
        "Sharpe Ratio": sharpe,
        "CAGR": cagr,
        "Volatility": volatility,
        "Max Drawdown": max_dd,
        "Final Balance": account_balance
    }

if __name__ == "__main__":
    main()

Fetching funding rate data...
Fetching historical Bitcoin price data...
Failed to fetch Bitcoin price data: 401
Using synthetic Bitcoin price data as fallback
Data fetched. 365 data points available.

=== STARTING FUNDING ARBITRAGE BACKTEST ===
Time period: 2024-03-01 to 2025-03-01
Min Funding Rate: 0.0005
Min Trade Size: 0.01 BTC
Max Trade Size: 0.05 BTC
Stop Loss: 3.0%
Max Drawdown: 20.0%
Found 301 potential arbitrage opportunities

[New Position 0] Arbitrage Opportunity Found! Funding Rate Difference: -0.000667
Opened hedge positions: buy futures on Bybit and sell spot on Binance, Size: 0.01334 BTC ($852.43)

[Position 0] Target close time reached. Closing position.
Closed hedge positions: buy spot on Binance and sell futures on Bybit, Size: 0.01334 BTC ($866.40)
Position closed. Profit: $-27.36 (Funding: $0.57, Spot: $-13.97, Futures: $-13.97)
Insufficient balance for new position. Required: $1723.94, Available: $9972.64
Insufficient balance for new position. Required: $3276.75, Av

/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_30896/2623157458.py:485: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  funding_performance = results_df.groupby('funding_range')['total_profit'].agg(['mean', 'count'])
